In [3]:
import numpy as np
import pandas as pd
import sklearn as sk
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, mean_squared_error, r2_score
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor,RandomForestClassifier
from sklearn.neighbors import KNeighborsRegressor


## Data traitment

Collecte et traitement des données

In [10]:
# Collecte des données
X= pd.read_csv(r"X_train_filled.csv",index_col=0)
Y = pd.read_csv(r"..\\y_train.csv",index_col=0)
data_train = pd.merge(X, Y, left_index=True, right_index=True)
data_train

,load_forecast,coal_power_available,gas_power_available,nucelear_power_available,wind_power_forecasts_average,solar_power_forecasts_average,wind_power_forecasts_std,solar_power_forecasts_std,predicted_spot_price,spot_id_delta
DELIVERY_START,,,,,,,,,,
2022-01-01 02:00:00+01:00,49439.0,3386.0,11487.0,44118.0,3035.0,0.0,79.248348,0.000000,135.062851,-36.874770
2022-01-01 03:00:00+01:00,46511.0,3386.0,11487.0,44118.0,3143.0,0.0,61.776532,0.000000,110.893448,-12.643588
2022-01-01 04:00:00+01:00,45158.0,3386.0,11487.0,44118.0,3288.0,0.0,44.291112,0.000000,82.303596,-1.950193
2022-01-01 05:00:00+01:00,44779.0,3386.0,11487.0,44118.0,3447.0,0.0,36.127588,0.000000,79.969795,1.938272
2022-01-01 06:00:00+01:00,45284.0,3386.0,11487.0,44118.0,3679.0,0.0,30.983023,0.000000,76.075630,0.199907
...,...,...,...,...,...,...,...,...,...,...
2023-03-29 19:00:00+02:00,50814.0,3386.0,11952.0,38320.0,7552.0,651.0,247.408490,7.821622,108.110000,6.029303
2023-03-29 20:00:00+02:00,50628.0,3386.0,11952.0,38320.0,8338.0,109.0,155.795012,2.534054,125.660000,13.576177
2023-03-29 21:00:00+02:00,48201.0,3386.0,11952.0,38320.0,9115.0,0.0,126.884684,0.000000,138.010000,17.478945


In [13]:
# Traitement
## Missing Data
data = data_train.dropna(axis=1, thresh=len(data_train)*0.25)
data = data.dropna()
#data = data.ffill()
X = data[[col for col in data.columns if col !='spot_id_delta']]
Y = data['spot_id_delta'].to_numpy()
## Format
X = pd.get_dummies(X)
## Split
X_train, X_test, y_train, y_test = sk.model_selection.train_test_split(X, Y, test_size=0.2, random_state=42)

## Model Selection

In [15]:
scalers = [None,StandardScaler(), MinMaxScaler(), RobustScaler()]
models = {
    'XGBoost' : XGBRegressor(),
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(),
    'Lasso Regression': Lasso(),
    'ElasticNet': ElasticNet(),
    'Support Vector Regression': SVR(),
    'Decision Tree Regression': DecisionTreeRegressor(),
    'Random Forest Regression': RandomForestRegressor(),
    'Gradient Boosting Regression': GradientBoostingRegressor(),
    #'K-Neighbors Regression': KNeighborsRegressor()
    }

res = list()
for mod in models.values() :
    print(f"Model: {mod.__class__.__name__}")
    for scaler in scalers:
        # Appliquer le scaler
        if scaler is None :
            X_train_scaled = X_train
            X_test_scaled = X_test
        else :
            X_train_scaled = scaler.fit_transform(X_train)
            X_test_scaled = scaler.transform(X_test)
            
        # Initialiser le modèle
        model = mod
        
        # Entraîner le modèle de base
        model.fit(X_train_scaled, y_train)
        
        # Évaluation du modèle
        y_pred = model.predict(X_test_scaled)
        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        # Validation croisée
        cv_scores = sk.model_selection.cross_val_score(model, X_train_scaled, y_train, cv=10, scoring='neg_mean_squared_error')
        res.append([mod.__class__.__name__,scaler.__class__.__name__,mse,r2,cv_scores])


Model: XGBRegressor
Scaler: NoneType
Scaler: StandardScaler
Scaler: MinMaxScaler
Scaler: RobustScaler
Model: LinearRegression
Scaler: NoneType
Scaler: StandardScaler
Scaler: MinMaxScaler
Scaler: RobustScaler
Model: Ridge
Scaler: NoneType
Scaler: StandardScaler
Scaler: MinMaxScaler
Scaler: RobustScaler
Model: Lasso
Scaler: NoneType
Scaler: StandardScaler
Scaler: MinMaxScaler
Scaler: RobustScaler
Model: ElasticNet
Scaler: NoneType
Scaler: StandardScaler
Scaler: MinMaxScaler
Scaler: RobustScaler
Model: SVR
Scaler: NoneType
Scaler: StandardScaler
Scaler: MinMaxScaler
Scaler: RobustScaler
Model: DecisionTreeRegressor
Scaler: NoneType
Scaler: StandardScaler
Scaler: MinMaxScaler
Scaler: RobustScaler
Model: RandomForestRegressor
Scaler: NoneType
Scaler: StandardScaler
Scaler: MinMaxScaler


KeyboardInterrupt: 

In [16]:
res = pd.DataFrame(res,columns=["model","scaler","mse","r2","cv_score"]).sort_values(by="r2",ascending=False)
res["cv_mean"] = res['cv_score'].apply(lambda x : np.mean(x))
res

,model,scaler,mse,r2,cv_score,cv_mean
29,RandomForestRegressor,StandardScaler,355.142415,0.575276,"[-2907.5429408649493, -303.55224488123986, -30...",-976.551999
28,RandomForestRegressor,NoneType,369.831579,0.557709,"[-3060.0802914609417, -313.8883829777885, -309...",-1014.308511
1,XGBRegressor,StandardScaler,459.238568,0.450784,"[-2132.732669303874, -399.8495642356061, -345....",-828.944717
0,XGBRegressor,NoneType,459.238568,0.450784,"[-2132.732669303874, -399.8495642356061, -345....",-828.944717
2,XGBRegressor,MinMaxScaler,459.238568,0.450784,"[-2132.732669303874, -399.8495642356061, -345....",-828.944717
3,XGBRegressor,RobustScaler,459.238568,0.450784,"[-2132.732669303874, -399.8495642356061, -345....",-828.944717
27,DecisionTreeRegressor,RobustScaler,632.013299,0.244158,"[-6438.997953865587, -599.3136373892922, -740....",-1901.122822
21,SVR,StandardScaler,822.436640,0.016426,"[-3492.1340347325568, -631.448527591158, -563....",-1358.351439
23,SVR,RobustScaler,824.866853,0.013520,"[-3497.729740858023, -633.3394908688516, -567....",-1363.098767
22,SVR,MinMaxScaler,828.942556,0.008645,"[-3501.223046387849, -635.0681048036364, -570....",-1365.467398


## XGBRegressor

Optimisation des paramètres

In [17]:
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

grid_search = sk.model_selection.GridSearchCV(estimator=RandomForestRegressor(), param_grid=param_grid, cv=3, scoring='r2', verbose=2, n_jobs=-1)
grid_search.fit(X_train_scaled, y_train)
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

Fitting 3 folds for each of 216 candidates, totalling 648 fits


In [18]:
y_pred_best = best_model.predict(X_test_scaled)
best_mse = mean_squared_error(y_test, y_pred_best)
best_r2 = r2_score(y_test, y_pred_best)

print(f"Best Mean Squared Error: {best_mse}")
print(f"Best R^2 Score: {best_r2}")

X_scaled = scaler.fit_transform(X)
best_cv_scores = sk.model_selection.cross_val_score(best_model, X_scaled, Y, cv=5, scoring='neg_mean_squared_error')
print(f"Best Cross-Validation Scores (neg MSE): {best_cv_scores}")
print(f"Mean Best Cross-Validation Score (neg MSE): {best_cv_scores.mean()}")

Best Mean Squared Error: 357.940579535313
Best R^2 Score: 0.5719293097278564
Best Cross-Validation Scores (neg MSE): [-1225.05364063 -2871.75430987 -2194.25747248  -870.76269147
  -550.4864703 ]
Mean Best Cross-Validation Score (neg MSE): -1542.4629169507873


In [19]:
best_params

{'bootstrap': True,
 'max_depth': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 50}

Creation et Validation du modèle

In [20]:
best_model = XGBRegressor(**best_params)

# Entraîner le modèle sur les données d'entraînement complètes
best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)

# Calcul des métriques
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Mean Squared Error: {mse}")
print(f"R2 Score: {r2}")

c:\Users\collo\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:160: UserWarning: [00:02:12] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "bootstrap", "min_samples_leaf", "min_samples_split" } are not used.

  warnings.warn(smsg, UserWarning)


Mean Squared Error: 490.2242311726361
R2 Score: 0.41372776090759456


In [21]:
best_model.fit(X_test,y_test)

c:\Users\collo\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:160: UserWarning: [00:02:17] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "bootstrap", "min_samples_leaf", "min_samples_split" } are not used.

  warnings.warn(smsg, UserWarning)


XGBRegressor(base_score=None, booster=None, bootstrap=True, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, min_samples_leaf=1, min_samples_split=2,
             missing=nan, monotone_constraints=None, multi_strategy=None,
             n_estimators=50, ...)

On utilie le model sur les valeurs manquantes

In [ ]:
X_test = pd.read_csv(r"..\\X_test.csv")

In [22]:
X_test = pd.get_dummies(X_test)
y_test = best_model.predict(X_test)

In [34]:
Y_test = pd.Series(y_test,index = X_test.index)
Y_test.name = "spot_id_delta"
Y_test.to_csv("Y test.csv")